# Map Generation

This project implements a random dungeon generator, based on [a paper](https://arxiv.org/pdf/1905.09618.pdf) by Daniel Ashlock and  Christoph Salge. Dungeon maps are generated via self-driving automata. The goal is to optimize the automata to generate compact maps. Optimization is done through a genetic algorithm.

In [1]:
# Project dependencies
import png
import random

The following classes define, respectively, a single room and a whole map/dungeon. A "room" can be a hallway (width or height equal to 1) or a normal room (width and height both greater than 1).

`draw_map` saves the map to a .png file. It could be modified to export to a file format more suitable for usage in a game.

At this time, there are no explicit doors or overall entrance/exit to the map.

The `evaluate` function is how we judge the fitness of a map. By changing the return value from `evalulate_compact` to `evaluate_sprawl`, the genetic algorithm will select for spread out maps rather than compact maps. Feel free to experiment with other custom functions.

In [2]:
# A single room in the map
class Room:
    def __init__(self,l,r,b,t):
        self.border = [l,r,b,t]
    # Check for overlap
    # Return True if no overlap, False if overlap
    def check_conflict(self, new_room):
        return self.border[0] >= new_room.border[1] or self.border[1] <= new_room.border[0] or self.border[2] >= new_room.border[3] or self.border[3] <= new_room.border[2]

class Map:
    def __init__(self,rooms):
        self.rooms = rooms

    # Get the range (far left, right, bottom, top) that the map covers
    def get_envelope(self):
        if len(self.rooms) == 0:
            return [0,0,0,0] # Should not happen because there should be at least one room by construction
        else:
            l = min([self.rooms[i].border[0] for i in range(len(self.rooms))])
            r = max([self.rooms[i].border[1] for i in range(len(self.rooms))])
            b = min([self.rooms[i].border[2] for i in range(len(self.rooms))])
            t = max([self.rooms[i].border[3] for i in range(len(self.rooms))])
            return [l,r,b,t]
    def get_area(self):
        return sum([(self.rooms[i].border[1]-self.rooms[i].border[0])*(self.rooms[i].border[3]-self.rooms[i].border[2]) for i in range(len(self.rooms))])

    # Evaluation functions

    # Favor compact maps with lots of rooms. This is the evaluation function used in the paper.
    def evaluate_compact(self):
        # Calculate a score of the fitness of the map
        env = self.get_envelope()
        return self.get_area()**2 / float((env[1]-env[0])*(env[3]-env[2]))

    # Favor sprawling maps
    def evaluate_sprawl(self):
        env = self.get_envelope()
        return (env[1]-env[0])*(env[3]-env[2])

    # Overall evaluations function
    def evaluate(self):
        # Change depending on what kind of maps we want to build
        return self.evaluate_compact()

    # Same the map to a file
    def draw_map(self,filename):
        envelope = self.get_envelope()
        if envelope[1]-envelope[0] > 100 or envelope[3]-envelope[2] > 100:
            print("Too big")
            return
        pixels = [[255]*(30*(envelope[1]-envelope[0])) for i in range(10*(envelope[3]-envelope[2]))]

        # Set up the grid
        for i in range(envelope[3]-envelope[2]):
            for j in range(envelope[1]-envelope[0]):
                for k in range(10):
                    pixels[10*i+0][30*j+0+3*k] = 0
                    pixels[10*i+0][30*j+1+3*k] = 0
                    pixels[10*i+0][30*j+2+3*k] = 0

                    pixels[10*i+9][30*j+0+3*k] = 0
                    pixels[10*i+9][30*j+1+3*k] = 0
                    pixels[10*i+9][30*j+2+3*k] = 0

                    pixels[10*i+k][30*j+0] = 0
                    pixels[10*i+k][30*j+1] = 0
                    pixels[10*i+k][30*j+2] = 0

                    pixels[10*i+k][30*j+27] = 0
                    pixels[10*i+k][30*j+28] = 0
                    pixels[10*i+k][30*j+29] = 0

        # Draw the rooms
        for i in range(len(self.rooms)):
            minx = 10*(self.rooms[i].border[0] - envelope[0])+1
            maxx = 10*(self.rooms[i].border[1] - envelope[0])-1
            miny = 10*(self.rooms[i].border[2] - envelope[2])+1
            maxy = 10*(self.rooms[i].border[3] - envelope[2])-1
            for x in range(minx,maxx):
                for y in range(miny,maxy):
                    if i == 0:
                        pixels[y][3*x+0] = 255
                        pixels[y][3*x+1] = 0
                        pixels[y][3*x+2] = 0
                    elif (self.rooms[i].border[1] == self.rooms[i].border[0]+1 or self.rooms[i].border[3] == self.rooms[i].border[2]+1):
                        pixels[y][3*x+0] = 0
                        pixels[y][3*x+1] = 255
                        pixels[y][3*x+2] = 0
                    else:
                        pixels[y][3*x+0] = 0
                        pixels[y][3*x+1] = 0
                        pixels[y][3*x+2] = 255
        # Set up for writing the file
        for i in range(len(pixels)):
            pixels[i] = tuple(pixels[i])
        with open(filename,"wb") as f:
            w = png.Writer(10*(envelope[1]-envelope[0]), 10*(envelope[3]-envelope[2]))
            w.write(f, pixels)


The following class defines a self-driving automaton, which generate the bits which in turn generates the map. I was not able to figure out how to program this so that the SDA example in the paper reproduces the output claimed (the example in the paper also has two out-arrows from a single node, so I think that's a mistake).

SDAs are highly expressive, in that they can generate complex, non-periodic bit strings. However, the resulting bit strings and consequently the scores of resulting maps are apparently not so sensitive to small changes in the SDA that it is possible to perform local optimization. Thus the SDA offers an effective compromise between the need to explore parameter space and the need to perform local optimization.

It would be interesting to quantify the above statements, perhaps by running some experiments to see how sensitive scores generally are to mutations (defined below) and how well map scores are "inherited" from the crossover operation.

In [3]:
# As described in the paper, maps are generated by a self-driving automaton.
# Note: I was not able to figure out how to program this so that the SDA example in the paper
# reproduces the output claimed.

class SDA:
    def __init__(self, labels, transitions):
        self.emit = labels
        self.next_state = transitions
        self.reset()

    # reset should be called every time a new map is generated.
    def reset(self):
        # Initialize parameters for producing bits
        self.bitstring = self.emit[0]
        self.num_bits = 0 # Number of bits produced so far. Grand total
        self.cur_bit = 0 # Number of bits produced so far on the current string

    # Feed the current output string back into the SDA to generate a longer output string.
    def expand_string(self):
        cur_node = 0
        new_string = self.emit[cur_node]
        for i in range(len(self.bitstring)):
            cur_node = self.next_state[cur_node][int(self.bitstring[i])]
            new_string = new_string + self.emit[cur_node]
        self.bitstring = new_string

    # Return a single bit (0 or 1), expanding the output string if necessary.
    def get_bit(self):
        if self.cur_bit >= len(self.bitstring):
            self.cur_bit = 0 # Not clear whether we want this line
            self.expand_string()
        bit = self.bitstring[self.cur_bit]
        self.cur_bit += 1
        self.num_bits += 1
        return int(bit)

    # Get multiple bits. This is the main interface for map generation.
    def get_bits(self,n):
        return [self.get_bit() for i in range(n)]


The following is the `Generator` class, which defines how maps are created from SDAs (or whatever other generation method is used, such as the Random method).

The `generate_map` function gathers bits from the SDAs (or whatever other source is specified) and translates them into rooms for the map. The fifth step, which specifies how a new room should be offset relative to the room to which it is attached, was not specified in the paper; either the authors left it out or I misunderstood it somehow.

`generate_map` offers endless possibility for variations and experimentation in how maps are generated.

In [4]:
# Generator class

# Hyperparameters related to  map generation.
num_room_attempts = 100 # Number of times we try to place a room in generating the map.

# gen is the generation object, mostly likely an SDA.
# method is either "SDA" or "Random".
# If method is "Random", gen is ignored and bits are generated randomly.
# If method is "SDA", then gen is the SDA which generates bits for the map.
class Generator:
    def __init__(self, gen, method):
        self.gen = gen
        self.method = method

    # Get multiple bits for map generation
    def get_bits(self, n):
        if self.method == "SDA":
            return self.gen.get_bits(n)
        else:
            return [random.randint(0,1) for i in range(n)]

    # A wrapper around get_bits that coverts to an integer from 0 to 2^n-1
    def get_num(self,n):
        bits = self.get_bits(n)
        return sum([bits[i]*2**i for i in range(len(bits)) ])

    # Make the map from the generator's string of bits.
    def generate_map(self):

        # Reset the SDA. Otherwise it will generate the wrong bits.
        if self.method == "SDA":
            self.gen.reset()

        # Initialize the rooms. Note that the Map object is not created until the end.
        rooms = [0]*num_room_attempts
        rooms[0] = Room(-2,2,-2,2) # Initial room.
        num_rooms = 1

        # Attempt to add rooms. num_room_attempts is only the theoretical maximum number of rooms that will result.
        for i in range(num_room_attempts):
            # Step 1 in Part B.
            base_room = self.get_num(8) % num_rooms

            # Step 2 in Part B
            # [0,0] is left, [1,0] is right, [0,1] is down, [1,1] is up
            next_direction = self.get_bits(2)

            # Step 3 in Part B
            is_corridor = (self.get_num(3) == 0)

            # Step 4 in Part B
            dims = [0,0] # [Horizontal, Vertical]
            if (is_corridor and not next_direction[1]):
                dims = [max(4,self.get_num(4)),1]
            if (is_corridor and next_direction[1]):
                dims = [1,max(4,self.get_num(4))]
            if (not is_corridor):
                dims = [max(2,1+self.get_num(2)),max(2,1+self.get_num(2))]

            # Step 5: Offset
            # I don't see this described in the paper.
            offset_num = self.get_num(4)
            if (not next_direction[1]):
                offset_min, offset_max = 1-dims[1], rooms[base_room].border[3]-rooms[base_room].border[2]
            if (next_direction[1]):
                offset_min, offset_max = 1-dims[0], rooms[base_room].border[1]-rooms[base_room].border[0]
            offset = offset_num % (offset_max-offset_min) + offset_min

            # With all the parameters set, figure out the location of the new room.
            next_border = [0,0,0,0]
            if (not next_direction[1]):
                next_border[2] = rooms[base_room].border[2]+offset
                next_border[3] = next_border[2] + dims[1]
                if (not next_direction[0]):
                    next_border[1] = rooms[base_room].border[0]
                    next_border[0] = next_border[1]-dims[0]
                else:
                    next_border[0] = rooms[base_room].border[1]
                    next_border[1] = next_border[0]+dims[0]
            if (next_direction[1]):
                next_border[0] = rooms[base_room].border[0]+offset
                next_border[1] = next_border[0] + dims[0]
                if (not next_direction[0]):
                    next_border[3] = rooms[base_room].border[2]
                    next_border[2] = next_border[3]-dims[1]
                else:
                    next_border[2] = rooms[base_room].border[3]
                    next_border[3] = next_border[2]+dims[1]
            new_room = Room(next_border[0], next_border[1], next_border[2], next_border[3])

            # Check if there is a conflict with previous rooms
            is_overlap = False
            for j in range(num_rooms):
                if not rooms[j].check_conflict(new_room):
                    is_overlap = True

            # Add the new room only if there is no conflict.
            if not is_overlap:
                rooms[num_rooms] = new_room
                num_rooms += 1
        return Map(rooms[:num_rooms])

    # Score by generating a map and scoring
    # Should not be used if method is "Random", as the score will not be consistent.
    def evaluate(self):
        map_ = self.generate_map()
        return map_.evaluate()

    # Draw a map by generating it first
    def draw_map(self, filename):
        self.generate_map().draw_map(filename)


The following defines our `Population` class, which maintains a population of maps and performs evolution. Feel free to experiment with the hyperparameters. Also, `random_sda` and `mutate` are constructed so nodes with length 1 and length 2 labels are equally likely; that assumption (or even the assumption that cells emit strings of lengths only 1 and 2) can be modified as well.

`crossover` implements two-point crossover. Other forms of [crossover](https://en.wikipedia.org/wiki/Crossover_(genetic_algorithm)) can be experimented with.

In [5]:
# Project hyperparameters

debug_mode = 1 # Set if you want to see progress displayed while running the evolutionary algorithm

# Hyperparameters related to the learning algorithm
mnm = 1 # Maximum number of mutations
sda_size = 12 # Number of nodes in each SDA
population_size = 32 # Number of SDAs kept in a population
tournament_size = 7 # Number of SDAs chosen for mating, of which the two best are crossed over.
generations = 10000 # Number of iterations of the genetic algorithm.

class Population:
    def __init__(self):
        self.pop = [self.random_sda() for i in range(population_size)]
        self.scores = [Generator(self.pop[i],"SDA").evaluate() for i in range(population_size)]

    # Create random SDAs for starting the algorithm
    def random_sda(self):
        labels = [["1","1","0","0","00","11","01","10"][random.randint(0,7)] for i in range(sda_size)]
        transitions = [[random.randint(0,sda_size-1),random.randint(0,sda_size-1)] for i in range(sda_size)]
        return SDA(labels,transitions)

    # Mutate an SDA
    def mutate(self, s):
        # Introduce random mutations to s
        num_mutations = random.randint(1,mnm)
        for i in range(num_mutations):
            random_node = random.randint(0,sda_size-1)
            if random.randint(0,1):
                s.emit[random_node] = ["1","1","0","0","00","11","01","10"][random.randint(0,7)]
            else:
                s.next_state[random_node][random.randint(0,1)] = random.randint(0,sda_size-1)

    # Crossover function. As per the paper, we are using two-point crossover.
    def crossover(self,s1,s2):
        # Find crossover points
        point1 = random.randint(0,sda_size-1)
        point2 = (point1+random.randint(1,sda_size-1))%sda_size
        if point1 > point2:
            point1, point2 = point2, point1

        # Perform the crossover
        emit3 = s1.emit[:point1]+s2.emit[point1:point2]+s1.emit[point2:]
        emit4 = s2.emit[:point1]+s1.emit[point1:point2]+s2.emit[point2:]
        # The following is a bit clunky because we need to deepcopy these arrays
        next_state3 = [0]*sda_size
        next_state4 = [0]*sda_size
        for i in range(point1):
            next_state3[i] = [s1.next_state[i][0], s1.next_state[i][1]]
            next_state4[i] = [s2.next_state[i][0], s2.next_state[i][1]]
        for i in range(point1, point2):
            next_state3[i] = [s2.next_state[i][0], s2.next_state[i][1]]
            next_state4[i] = [s1.next_state[i][0], s1.next_state[i][1]]
        for i in range(point2, sda_size):
            next_state3[i] = [s1.next_state[i][0], s1.next_state[i][1]]
            next_state4[i] = [s2.next_state[i][0], s2.next_state[i][1]]

        # Build new SDAs
        s3 = SDA(emit3, next_state3)
        s4 = SDA(emit4, next_state4)

        # Mutation
        self.mutate(s3)
        self.mutate(s4)

        return s3,s4

    # Perform a single cycle of the genetic algorithm.
    def update(self):
        selection = random.sample(range(population_size),tournament_size)

        # Find the best two and worst two
        best = selection[0]
        second_best = selection[1]
        worst = selection[0]
        second_worst = selection[1]
        if self.scores[best] < self.scores[second_best]:
            best,second_best = second_best,best
        if self.scores[worst] > self.scores[second_worst]:
            worst,second_worst = second_worst,worst
        for i in range(2,tournament_size):
            if self.scores[selection[i]] >= self.scores[best]:
                best,second_best = selection[i],best
            elif self.scores[selection[i]] >= self.scores[second_best]:
                second_best = selection[i]
            if self.scores[selection[i]] <= self.scores[worst]:
                worst,second_worst = selection[i],worst
            elif self.scores[selection[i]] <= self.scores[second_worst]:
                second_worst = selection[i]

        # Mate them and replace the worst ones
        new1, new2 = self.crossover(self.pop[best], self.pop[second_best])
        self.pop[worst] = new1
        self.pop[second_worst] = new2
        self.scores[worst] = Generator(new1,"SDA").evaluate()
        self.scores[second_worst] = Generator(new2,"SDA").evaluate()

    # Perform the full genetic algorithm.
    def evolve(self):
        if debug_mode:
            print( "Initial Population: Max score is " + str(max(self.scores)) )
        for i in range(generations):
            if debug_mode and i%100 == 0:
                print( "Generation " + str(i) + ": Max score is " + str(max(self.scores)) )
            self.update()

    # Return the best specimen as an SDA.
    # Meant to be called only after the genetic algorithm has been run.
    def get_best(self):
        return self.pop[self.scores.index(max(self.scores))]

    # Draw the map from the best SDA.
    def draw_map(self, filename):
        Generator(self.get_best(),"SDA").draw_map(filename)


In [6]:
# Now run the full algoritm.
# Warning: it may take a while.
# The output is saved to a file.
pop = Population()
pop.evolve()
pop.draw_map("map.png")

# Uncomment the following to draw a random map
#gen = Generator(None, "Random")
#gen.draw_map("random.png")

Initial Population: Max score is 185.640625
Generation 0: Max score is 185.640625


Running the evolutionary algorithm 10 times and taking the best specimen from each run, we got the following scores. Of these runs, only one (the fourth) produced a map with hallways, and there was one.

- 244.28070175438597
- 222.17142857142858
- 230.7983682983683
- 247.390243902439
- 232.96258847320524
- 242.3063063063063
- 223.32142857142858
- 252.2840909090909
- 243.50935550935552
- 237.18861607142858

The following are deciles from the scores of 1000 randomly generated SDAs.

- 8.680555555555555
- 15.0
- 24.0
- 35.84848484848485
- 48.390243902439025
- 59.73011363636363
- 68.82563025210084
- 81.1298076923077
- 94.6125
- 111.36011477761836
- 191.14666666666668

The following are deciles from the scores of 1000 maps from randomly generated bits (not using an SDA).

- 33.96450304259635
- 60.12687687687688
- 67.50852272727273
- 72.63529411764706
- 76.82671957671958
- 81.93939393939394
- 86.60336134453782
- 92.24491600353669
- 99.54092071611254
- 110.50096153846154
- 162.6576086956522